In [ ]:
import itertools
import os

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import numpy as np
import open3d as o3d
import pandas as pd
from scipy import spatial
from scipy import interpolate
from sklearn.decomposition import PCA
from sklearn import preprocessing

from utils import normals_to_rgb
from plotting import set_axes_equal, set_defense_context, draw_unit_cube

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
def add_coordinate_frame(ax):
    """Set RGB coordinate frame to axes.
    
    Parameters
    ----------
    ax : matplotlib.axes._subplots.Axes3DSubplot
        3-D axes subplot.
    
    Returns
    -------
    matplotlib.axes._subplots.Axes3DSubplot
        Axes with coordinate frame.
    """
    ax.quiver(-1.5, -1, -1.5, 0.75, 0, 0, color='r')
    ax.text(0, -1, -1.5, s='$x$', color='r', fontweight='bold')
    ax.quiver(-1.5, -1, -1.5, 0, 0.75, 0, color='g')
    ax.text(-1.5, +0.25, -1.5, s='$y$', color='g', fontweight='bold')
    ax.quiver(-1.5, -1, -1.5, 0, 0, 0.75, color='b')
    ax.text(-1.5, -1, -0.5, s='$z$', color='b', fontweight='bold')
    ax.scatter(-1.5, -1, -1.5, color='k', depthshade=False)
    return ax

# PCA via Open3D - unit normal

In [ ]:
# load ear coordinates

xyz = pd.read_csv(os.path.join('data', 'ear.xyz')).values * 100  # in cm
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)

In [ ]:
# estimate normals by using open3d

pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamKNN(knn=111),
    fast_normal_computation=True
)
pcd.normalize_normals()
pcd.orient_normals_consistent_tangent_plane(k=33)
n_o3d = np.asarray(pcd.normals)

In [ ]:
# downsampling

pcd_ds = pcd.voxel_down_sample(voxel_size=0.05)
xyz_ds = np.asarray(pcd_ds.points)
n_o3d_ds = np.asarray(pcd_ds.normals)

In [ ]:
# visualize

with set_defense_context():
    fig = plt.figure(figsize=(5, 5))
    ax = plt.axes(projection ='3d')
    ax.scatter(*xyz_ds.T, c='k', alpha=0.1, s=0.1)
    ax.quiver(*xyz_ds.T, *n_o3d_ds.T, color='k',
              length=0.5, lw=0.5, alpha=0.5)
    ax.view_init(20, 150)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    fig.tight_layout()
    plt.show()
    fname = '04-anatomical-ear-model-quiver.png'
    # fig.savefig(os.path.join('figures', fname), dpi=300)

In [ ]:
# convert arrows to rgb cube

c_o3d = normals_to_rgb(n_o3d)

with set_defense_context():
    fig = plt.figure(figsize=(5, 5))
    ax = plt.axes(projection ='3d')
    s = ax.scatter(*xyz.T, color=c_o3d, s=0.3)
    ax = add_coordinate_frame(ax)
    ax.view_init(20, 155)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    fig.tight_layout()
    plt.show()
    fname = '04-anatomical-ear-model-rgb-pca.png'
    # fig.savefig(os.path.join('figures', fname), dpi=300)

# Spline - curvature normal

In [ ]:
# estimate normals manually

n_spline = np.empty_like(xyz)
tree = spatial.KDTree(xyz)
for i, query_point in enumerate(xyz):
    nbh_dist, nbh_idx = tree.query([query_point], k=111)
    query_nbh = xyz[nbh_idx.flatten()]

    X = query_nbh.copy()
    X_norm = X - X.mean(axis=0)
    U, S, VT = np.linalg.svd(X_norm.T)
    X_trans = X_norm @ U

    h = interpolate.SmoothBivariateSpline(*X_trans.T)

    cn = np.array([-h(*X_trans[0, :2], dx=1).item(),
                   -h(*X_trans[0, :2], dy=1).item(),
                   1])
    cn = cn.T @ U.T
    un = np.divide(cn, np.linalg.norm(cn, 2))
    n_spline[i, :] = un

In [ ]:
# load ear coordinates and orient normals

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
pcd.normals = o3d.utility.Vector3dVector(n_spline)
pcd = pcd.normalize_normals()
pcd.orient_normals_consistent_tangent_plane(k=111)
n_spline = np.asarray(pcd.normals)

In [ ]:
# convert arrows to rgb cube

c_spline = normals_to_rgb(n_spline)

with set_defense_context():
    fig = plt.figure(figsize=(5, 5))
    ax = plt.axes(projection ='3d')
    s = ax.scatter(*xyz.T, color=c_spline, s=0.3)
    ax = add_coordinate_frame(ax)
    ax.view_init(20, 155)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    fig.tight_layout()
    plt.show()
    fname = '04-anatomical-ear-model-rgb-spline.png'
    # fig.savefig(os.path.join('figures', fname), dpi=300)

# Error angle

In [ ]:
# compute error

theta = np.arccos(
    np.round(  # to avoid numerical instabilities
        np.abs(  # to neglect orientation
            np.sum(n_o3d * n_spline, axis=1)  # scalar product
        ), 6)) * 180 / np.pi
theta_rms = np.sqrt(np.mean(theta) ** 2)
theta_rms

In [ ]:
# histogram of errors

with set_defense_context():
    fig = plt.figure(figsize=(4, 3))
    ax = plt.axes()
    ax.hist(theta, bins=31, color='gray')
    ax.set(xlabel='theta (deg)', ylabel='count',
           title=f'rms = {theta_rms:.2f}°')
    fig.tight_layout()
    plt.show()

In [ ]:
# convert arrows to rgb cube

with set_defense_context():
    fig = plt.figure(figsize=(5, 5))
    ax = plt.axes(projection ='3d')
    s = ax.scatter(*xyz.T, c=theta, s=0.3)
    ax = add_coordinate_frame(ax)
    ax.view_init(20, 155)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    fig.colorbar(s, ax=ax, orientation='vertical', shrink=0.5, pad=-0.25,
                 label=r'$\theta$ (°)')
    fig.tight_layout()
    plt.show()
    fname = '04-angle-error.png'
    # fig.savefig(os.path.join('figures', fname), dpi=300)

# RGB cube

In [ ]:
# set up coloring for rgb cube

pts = np.array(list(itertools.product([0, 1], repeat=3)))
cs = ['black', 'blue', 'green', 'cyan', 'red', 'magenta', 'yellow', 'white']
pairs = pd.DataFrame(data=pts, columns=['x', 'y', 'z'])
pairs['cs'] = cs

In [ ]:
# visualize

with set_defense_context():
    fig = plt.figure(figsize=(2, 2))
    ax = plt.axes(projection ='3d')
    ax = draw_unit_cube(ax)
    ax.scatter(*pts.T, c=cs, edgecolor='k', depthshade=False, s=500)
    ax.view_init(20, 155)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    fig.tight_layout()
    plt.show()
    fname = '04-rgb-cube.png'
    # fig.savefig(os.path.join('figures', fname), dpi=300)